In [13]:
import torch

import torch.nn as nn

class NetRelu(nn.Module):
    def __init__(self,D_in,H1,H2,D_out):
        
        super(NetRelu,self).__init__()
        
        self.linear1=nn.Linear(D_in,H1)
        self.linear2=nn.Linear(H1,H2)
        self.linear3=nn.Linear(H2,D_out)
        
    def forward(self,x):
        x=torch.relu(self.linear1(x))
    
        x= torch.relu(self.linear2(x))
        
        x=self.linear3(x)
        
        return x

In [14]:
import torchvision

import torchvision.transforms as transforms

train_datasets=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transforms.ToTensor())

validation_dataset=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transforms.ToTensor())

In [15]:
train_loader=torch.utils.data.DataLoader(dataset=train_datasets,batch_size=2000)

validation_loader=torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=5000)

In [16]:
def train(model,criterion,train_loader,validation_loader,optimizer,epochs=100):
    i=0
    useful_stuff={'training_loss':[],'validation_accuracy':[]}
    
    for epoch in range(epochs):
        for i,(x,y) in enumerate(train_loader):
            
            optimizer.zero_grad()
            z=model(x.view(-1,28*28))
            loss=criterion(z,y)
            loss.backward()
            optimizer.step()
            useful_stuff['training_loss'].append(loss.data.item())
        correct=0
    
        for x,y in validation_loader:
            z=model(x.view(-1,28*28))
            _,label=torch.max(z,1)
            correct+=(label==y).sum().item()
        accuracy=100*(correct/len(validation_dataset))
    useful_stuff['validation_accuracy'].append(accuracy)
    return useful_stuff

In [17]:
model=NetRelu(784,50,50,10)
train_loader=torch.utils.data.DataLoader(dataset=train_datasets,batch_size=2000)

validation_loader=torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=5000)

criterion=nn.CrossEntropyLoss()

optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [18]:
training_results=train(model,criterion,train_loader,validation_loader,optimizer,epochs=30)